# Dataloading

In [8]:
!pip install -q torch torchvision matplotlib tqdm gdown
import shutil
import urllib.request
import os
if not os.path.exists("canopy_height_dataset.zip"):
    print(f"downloading Canopy height.zip")
    urllib.request.urlretrieve("https://drive.google.com/file/d/1sGr43Mo5s87K7mU9Jt3wlSeE-sg0_2gJ/view?usp=drive_link", "canopy_height_dataset.zip")



In [ ]:
#If you want to extract the whole dataset
import zipfile
with zipfile.ZipFile("canopy_height_dataset.zip", 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
# Or, if you just want to use one image+label
import zipfile
with zipfile.ZipFile("canopy_height_dataset.zip", 'r') as zip_ref:
    with zip_ref.open('/canopy_height_dataset/images/1.tif') as zf, open('/temp/images/1.tif','wb') as f:
        shutil.copyfileobj(zf, f)

#/canopy_height_dataset/images/1.tif
#/canopy_height_dataset/labels/window_1.tif

# Create a Class for the dataset
from exercise 7. We need to change the labels because we are doing a per pixel regression. Also, the images should already be separated in train/val/test

In [ ]:
from torch.utils.data import Dataset

from PIL import Image

import os
import glob


class UCMerced(Dataset):

    # mapping between label class names and indices
    LABEL_CLASSES = {
      'agricultural': 		  0,
      'airplane': 			    1,
      'baseballdiamond': 	  2,
      'beach': 				      3,
      'buildings': 			    4,
      'chaparral': 			    5,
      'denseresidential':   6,
      'forest': 				    7,
      'freeway': 				    8,
      'golfcourse': 			  9,
      'harbor': 				    10,
      'intersection': 		  11,
      'mediumresidential':  12,
      'mobilehomepark': 	  13,
      'overpass': 			    14,
      'parkinglot': 			  15,
      'river': 				      16,
      'runway': 				    17,
      'sparseresidential':  18,
      'storagetanks': 		  19,
      'tenniscourt': 			  20
    }

    # image indices to use for different splits
    SPLITS = {
      'train': list(range(0, 60)),    # use first 60 images of each class for training...
      'val':   list(range(61, 70)),   # ...images 61-70 for model validation...
      'test':  list(range(71, 100))   # ...and the rest for testing
    }

    def __init__(self, transforms=None, split='train'):
        self.transforms = transforms

        # prepare data
        self.data = []  # list of tuples of (image path, label class)
        for labelclass in self.LABEL_CLASSES:
            # get images with correct index according to dataset split
            for imgIndex in self.SPLITS[split]:
                imgName = os.path.join('UCMerced_LandUse/Images', labelclass, f'{labelclass}{str(imgIndex).zfill(2)}.tif')
                # example format: 'baseFolder/agricultural/agricultural07.tif'
                self.data.append((
                    imgName,
                    self.LABEL_CLASSES[labelclass]          # get index for label class
                ))


    #TODO: please provide the remaining functions required for the torch.utils.data.Dataset class.
    def __len__(self):
        return len(self.data)


    def __getitem__(self, x):
        imgName, label = self.data[x]

        img = Image.open(imgName)
        if self.transforms is not None:
            img = self.transforms(img)
        return img, label

## Extract an image
From exercise 7, need to change it

In [ ]:
import matplotlib.pyplot as plt

# initialize the dataset (call the constructor __init__)
dataset = UCMerced()
print(f"dataset of length {len(dataset)}")

# plot individual samples
from ipywidgets import interact
@interact(idx=range(len(dataset)))
def plot_sample(idx=0):
    img, label = dataset[idx]

    plt.imshow(img)

    # swaps keys and values in the dictionary UCMerced.LABEL_CLASSES
    class_mapping = {v: k for k, v in UCMerced.LABEL_CLASSES.items()}

    plt.title(f"classid {label} ({class_mapping[label]})")

# Define the model
Here is an hypercolumn from exercise 9 but we need to use a UNet.

In [ ]:
import os.path
import torch.nn as nn

# Define convolutions
def conv3x3(in_channels, out_channels, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=True, dilation=dilation)


def conv1x1(in_channels, out_channels, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=1, stride=stride, bias=True)



## Define model class

# Hypercolumn model from ex9 TO CHANGE
class Hypercolumn(nn.Module):

    def __init__(self):
        super(Hypercolumn, self).__init__()

        #TODO: define your architecture and forward pass here
        self.block1 = nn.Sequential(
            nn.Conv2d(4, 32, kernel_size=5, stride=4),
            nn.MaxPool2d(kernel_size=2, stride=1),
            nn.BatchNorm2d(num_features=32),
            nn.ReLU(inplace=True)
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=4),
            nn.MaxPool2d(kernel_size=2, stride=1),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(inplace=True)
        )
        self.block3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=5, stride=2),
            nn.MaxPool2d(kernel_size=2, stride=1),
            nn.BatchNorm2d(num_features=128),
            nn.ReLU(inplace=True)
        )
        self.block4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1),
            nn.MaxPool2d(kernel_size=2, stride=1),
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(inplace=True)
        )
        self.final = nn.Sequential(
            nn.Conv2d(484, 256, kernel_size=1, stride=1),           # 485 = 256 + 128 + 64 + 32 + 4 (input bands)
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 6, kernel_size=1, stride=1)
        )
    

    def forward(self, x):
        #TODO
        upsample = nn.Upsample(size=(x.size(2), x.size(3)))
        x1 = self.block1(x)
        x2 = self.block2(x1)
        x3 = self.block3(x2)
        x4 = self.block4(x3)

        hypercol = torch.cat(
            (x, upsample(x1), upsample(x2), upsample(x3), upsample(x4)),
            dim=1
        )
        return self.final(hypercol)